# __Byproduct Recovery Framework Tutorial__

The purpose of this tutorial is to introduce the __byproduct recovery decision framework__ (hereafter referenced to as _the framework_), which is defined in ``determine_byproduct_recovery.py``.The framework assesses the economic viability of converting waste streams into saleable byproducts based on a known viability criterion. This framework serves as a decision-support tool for determining whether specific byproducts should be recovered. 

This tutorial gives an example of how to use the framework to determine if lithium and cobalt should be recovered from a diafiltration process. This framework can be generalized to other recoverable byproducts if appropriate pricing data is provided. 

To evaluate byproduct recovery, annualized net benefit is assessed with the following equations:
$$
\text{net\_benefit} = \text{potential\_revenue} - \text{total\_recovery\_cost}
$$

$$
\text{potential\_revenue} = \sum_{m} \left( 
\text{material\_production}_{m} \cdot \text{market\_value}_{m} + 
\text{avoided\_waste\_disposal\_cost}_{m} \right)
$$

$$
\text{total\_recovery\_cost} = \sum_{m} \left( 
\text{conversion\_cost}_{m} \cdot \text{conversion\_possible}_{m} + 
\text{added\_process\_cost}_{m} \cdot \text{added\_process\_steps}_{m} \right)
$$

**Definitions of Variables:**

- `material_productionₘ`: Mass flow rate of recovered byproduct *m* (kg/yr).
- `market_valueₘ`: Market price of byproduct *m* (USD/kg).
- `avoided_waste_disposal_costₘ`: Avoided disposal cost for byproduct *m* (USD/kg).
- `conversion_costₘ`: Annualized cost (considering both CAPEX and OPEX) of the recovery process for byproduct *m* (USD/yr).
- `conversion_possibleₘ`: Binary indicator for whether conversion is required for byproduct *m* (1 = required, 0 = not required).
- `added_process_costₘ`: Annualized cost (considering both CAPEX and OPEX) of additional process units for recovering byproduct *m* (USD/yr).
- `added_process_stepsₘ`: Binary indicator for whether addiitonal process units are required for recovering byproduct *m* (1 = required, 0 not required).

To use the byproduct recovery framework effectively, a byproduct recovery process with costing must already be constructed. To learn the basics of the REE Costing Framework, please refer to [Part 1 of the REE Costing Tutorial](ree_costing_framework_basic_features-solution.ipynb). This is necessary to determine the __annualized cost__ and __annual revenue__ associated with recovery. Once the process model is built, users can import it—along with its corresponding cost and revenue—and apply the byproduct recovery framework to decide whether or not the byproduct should be recovered.

__Learning Objectives__: By the end of this tutorial, users will be able to:
- Decide and import the necessary Python packages
- Build a process flowsheet using a case study: __lithium-cobalt recovery via diafiltration__
- Apply the __byproduct recovery decision framework__

## __Step 1: Import the Necessary Packages__

First, import the necessary Pyomo, IDAES, and PrOMMiS packages. These will be implemented at various stages throughout the tutorial.

### __Package Purposes__
- Pyomo: Used for model construction and optimization.

- IDAES: While not essential for this specific tutorial, IDAES models can support model testing and validation.

- PrOMMiS:

    - ``prommis.nanofiltration.diafiltration``: Used to build the lithium-cobalt diafiltration recovery process model.

    - ``prommis.uky.costing.costing_dictionaries``: Provides product pricing data.

    - ``prommis.uky.costing.determine_byproduct_recovery``: Implements the byproduct recovery decision framework.
 
For guidance on installing these packages, please refer to the Package Installation section at the end of the tutorial.

In [1]:
# Pyomo packages
from pyomo.environ import (
    ConcreteModel,
    Expression,
    Param,
    TransformationFactory,
    Var,
    value,
)

# IDAES packages
from idaes.core.util.model_diagnostics import DiagnosticsToolbox
from idaes.core.util.model_statistics import degrees_of_freedom

import pytest

# PrOMMiS packages
from prommis.nanofiltration.diafiltration import (
    main,
    add_costing,
    add_objective,
    add_product_constraints,
    build_model,
    initialize_model,
    print_information,
    set_scaling,
    solve_model,
    unfix_opt_variables,
)
from prommis.uky.costing.costing_dictionaries import load_default_sale_prices
from prommis.uky.costing.determine_byproduct_recovery import (
    ByproductRecovery,
    determine_example_usage,
)

## __Step 2: Building on the Pre-defined Diafiltration Process__

This section gives a brief introduction of how to build upon the pre-defined diafiltration process model. The lithium-cobalt diafiltration process is implemented in the file [diafiltration.py](https://github.com/prommis/prommis/blob/main/src/prommis/nanofiltration/diafiltration.py), available in the PrOMMiS repository. 

To construct a process flowsheet using this existing model, users can refer to the detailed walkthrough provided in the [Multi_Stream Contactor Tutorial (Solution)](https://github.com/prommis/prommis/blob/main/docs/tutorials/diafiltration-solution.ipynb). This tutorial outlines major order for building the flowsheet. 

To determine whether lithium and cobalt should be recovered, the user first needs to __build the diafiltration process__ and __access the cost and revenue__.

Refer the process model defined in ``diafiltration.py``. Specifically, call the functions in the order specified within the ``main()`` function. Following this order is crucial because the diafiltration process consists of three membrane stages, and the model is built stage by stage. 

During each stage of model building, initialization is performed with a set of variables fixed. When transitioning to the next stage, previously fixed variables must be unfixed, and new variables need to be fixed for the current stage. This sequential initialization ensures that the model is stable and properly configured at each membrane stage.

The production rates of lithium and cobalt from the diafiltration process are expressed in kilograms per hour (kg/hr). Assuming the plant operates 8,000 hours annually, the annual production of lithium and cobalt can be calculated by multiplying the hourly production rates by 8,000 hr/yr.

Note that the code may generate warnings related to volumetric flow being temporarily set to zero during initialization—values that fall outside the predefined variable bounds. These warnings do not affect model performance and can be safely ignored.

In [2]:
# Build and solve the model
m = build_model()
add_costing(m)
initialize_model(m)
solve_model(m)

unfix_opt_variables(m)
add_product_constraints(m, Li_recovery_bound=0.95, Co_recovery_bound=0.635)
add_objective(m)
set_scaling(m)
scaling = TransformationFactory("core.scale_model")
scaled_model = scaling.create_using(m, rename=False)
solve_model(scaled_model)
# Propagate results back to unscaled model
scaling.propagate_solution(scaled_model, m)

# Store results for later application in the framework
total_annualized_cost = value(m.fs.costing.total_annualized_cost)

# recovery mass flow rate annually: kg/hr * 8000 hr/yr
Li_recovery_mass = value(m.fs.stage3.permeate_outlet.flow_vol[0]) * value(
    m.fs.stage3.permeate_outlet.conc_mass_solute[0, "Li"] * 8000 
)
Co_recovery_mass = value(m.fs.stage1.retentate_outlet.flow_vol[0]) * value(
    m.fs.stage1.retentate_outlet.conc_mass_solute[0, "Co"] * 8000 
)

WARNING (W1002): Setting Var 'fs.stage1.permeate[0.0,1].flow_vol' to a numeric
value `0.0` outside the bounds (1e-08, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.stage1.permeate[0.0,2].flow_vol' to a numeric
value `0.0` outside the bounds (1e-08, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.stage1.permeate[0.0,3].flow_vol' to a numeric
value `0.0` outside the bounds (1e-08, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.stage1.permeate[0.0,4].flow_vol' to a numeric
value `0.0` outside the bounds (1e-08, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.stage1.permeate[0.0,5].flow_vol' to a numeric
value `0.0` outside the bounds (1e-08, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting 

## __Step 3: Test the Byproduct Recovery Framework__

Now that the model is solved, the next task is to calculate the __cost__ and evaluate __economic feasibility__. 

__3.1. Specify Product Prices__

Product prices are defined in [costing_dictionary.py](https://github.com/prommis/workspace/blob/main/prommis_workspace/UKy_flowsheet/costing/costing_dictionaries.py). Users can access the default sale prices via ``load_default_sale_prices()`` function, as shown below:

In [3]:
sale_prices = load_default_sale_prices()
Li_price = sale_prices["Li"]
Co_price = sale_prices["Co"]

Alternatively, users may specify custom sale prices by manually replacing the right-hand side values of the price expressions.

These prices will be used to compute __annual revenue__, which is a key input to the byproduct recovery decision framework.

__3.2. Introduce the Framework and Specify the Product Outputs__

Next, introduce the byproduct recovery framework and speficy the target products recovered from the diafiltration process.

The __byproduct recovery framework__ accepts user inputs, where products are listed in a structure called ``material_list``, and their associated data (e.g. flow rate, price) are organized in a dictionary called ``material_data``.
- ``material_list``: A list of product names (strings).
- ``material_data``: A dictionary where each key corresponds to a product name and maps to its properties such as flow rate and price.

This structured input will be passed into the byproduct recovery decision framework to assess economic viability.

In [4]:
# The framework take user input
material_list = ["Lithium", "Cobalt"]

# Introduce the byproduct recovery framework
model = ConcreteModel()
model.recovery_determine = ByproductRecovery(materials=material_list)

# Define input values based on provided materials
material_data = {
    "Lithium": {
        "production": Li_recovery_mass,
        "market_value": Li_price,
        "waste_disposal": 1,
        "conversion": 0,
        "conversion_cost": 0,
        "process_steps": 1,
        "process_cost": total_annualized_cost,
    },
    "Cobalt": {
        "production": Co_recovery_mass,
        "market_value": Co_price,
        "waste_disposal": 1,
        "conversion": 0,
        "conversion_cost": 0,
        "process_steps": 0,
        "process_cost": 0,
    },
}

__3.3. Set Additional Parameters and Evaluate Net Benefit__

To evaluate the economic feasibility, we must define additional parameters for each product:

- ``"waste_disposal"``: Set to ``1`` to indicate that waste disposal costs are considered.

- ``"conversion"``: Set to ``0`` if no conversion process is required to recover the material.

- ``"conversion_cost"``: Set to ``0`` when conversion is not needed.

- ``"process_steps"``: Set to ``0`` if no additional steps are needed beyond the current recovery process.

In this case, the lithium–cobalt diafiltration plant recovers __both lithium and cobalt in a single process__. No additional purification or recovery steps are considered for either product. As such:

- The __process cost__ is only accounted for once under ``"Lithium"`` production (though it could alternatively be placed under ``"Cobalt"`` with no impact on results).

- __Both products are assumed to be sold at market prices__, regardless of purity level.

In [5]:
# Set values based on material list
for m in material_list:
    data = material_data.get(m, {})
    model.recovery_determine.material_production[m].set_value(data.get("production", 0))
    model.recovery_determine.market_value[m].set_value(data.get("market_value", 0))
    model.recovery_determine.waste_disposal_cost[m].set_value(
        data.get("waste_disposal", 0)
    )
    model.recovery_determine.conversion_possible[m].set_value(data.get("conversion", 0))
    model.recovery_determine.conversion_cost[m].set_value(
        data.get("conversion_cost", 0)
    )
    model.recovery_determine.added_process_steps[m].set_value(
        data.get("process_steps", 0)
    )
    model.recovery_determine.added_process_cost[m].set_value(
        data.get("process_cost", 0)
    )

> 💡 **Note:** The `0` in `data.get("production", 0)` does not mean zero production. 
It indicates that the production rate is assumed to be **steady-state** (i.e., constant over time), not time-varying.

__3.4. Calculate the Overall Net Benefit__

With parameters such as production rate, market price, and associated costs specified for each product, the byproduct recovery framework can now be implemented to quantify the overall net benefit.

The __overall net benefit__ is used to determine profitability:

- If ``net_benefit_value`` > 0 → The byproduct recovery process is __profitable__.

- If ``net_benefit_value`` ≤ 0 → The process is __not profitable__.

In [6]:
potential_revenue = value(model.recovery_determine.potential_revenue)
assert potential_revenue >= 0, "Potential revenue should be non-negative."

determine_result = model.recovery_determine.determine_financial_viability()
assert isinstance(
    determine_result, str
), f"Expected a string message, but got {type(determine_result)}"

# Check the output string for financial viability
net_benefit_value = value(model.recovery_determine.net_benefit)
assert net_benefit_value == pytest.approx(320373532.81, rel=1e-4)

if net_benefit_value > 0:
    expected_message = f"✅ Byproduct recovery is financially viable. Net Benefit: {net_benefit_value:.2f} $/yr"
else:
    expected_message = f"❌ Byproduct recovery is NOT financially viable. Loss: {-net_benefit_value:.2f} $/yr"

assert determine_result == expected_message, f"Unexpected output: {determine_result}"

print("\n--- Byproduct Recovery Decision ---")
print(determine_result)


--- Byproduct Recovery Decision ---
✅ Byproduct recovery is financially viable. Net Benefit: 320373532.81 $/yr


> ✅ **Note:** The `assert` statements are included to verify that the model is built as expected.  
> They are helpful for testing and debugging, but **not required** for using the byproduct recovery framework.

## __Package Installation__

As mentioned previously, Pyomo, IDAES, and PrOMMiS packages are necessary to use the framework. If these packages are not installed, this maybe done using the following commands in Anaconda Prompt:

For more detailed installation instructions, refer to the respective ``README.md`` files for [Pyomo](https://github.com/Pyomo/pyomo/blob/main/README.md), [IDAES](https://github.com/IDAES/idaes-pse/blob/main/README.md), and [ProMMiS](https://github.com/prommis/prommis/blob/main/README.md).